In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
from urllib.request import urlopen

In [5]:
from bs4 import BeautifulSoup

In [6]:
import pandas as pd

In [7]:
import numpy as np

In [8]:
import random

In [30]:
# Load the draft odds and team data
draft_odds_path = 'Draft Odds 2022.csv'
nba_teams_path = 'NBA_Teams_Final_2022.csv'
player_stats_path = 'Sid_Offense_Polygence_Final.csv'


draft_odds = pd.read_csv(draft_odds_path)
nba_teams = pd.read_csv(nba_teams_path)
player_stats = pd.read_csv(player_stats_path)

# Convert percentages to probabilities
probabilities = {}
for index, row in draft_odds.iterrows():
    team = row['Team']
    probabilities[team] = []
    for pick in range(1, 15):
        value = row[f'{pick}']
     #   if value == '-' or pd.isna(value):
     #       prob = 0.0
     #   elif '<' in value:
     #       prob = 0.0001  # Use a small value for '<0.01'
     #   else:
        prob = float(value.strip('%')) / 100
        probabilities[team].append(prob)

# Get the fixed order for picks 15-30
fixed_order = draft_odds.iloc[14:30]['Team'].tolist()

# Define a function to simulate the lottery
def simulate_lottery(probabilities, num_simulations=1):
    results = {team: {pick: 0 for pick in range(1, 15)} for team in probabilities.keys()}
    
    for _ in range(num_simulations):
        remaining_teams = list(probabilities.keys())
        picks = []
        
        for pick in range(1, 15):
            if not remaining_teams:
                break

            # Get probabilities for the current pick
            probs = np.array([probabilities[team][pick - 1] for team in remaining_teams])
            
            # Check if the sum of probabilities is zero
            if probs.sum() == 0:
                # Assign equal probabilities if all are zero
                probs = np.array([1 / len(remaining_teams)] * len(remaining_teams))
            else:
                # Normalize probabilities to sum to 1
                probs = probs / probs.sum()
            
            # Make a weighted random choice
            chosen_team = np.random.choice(remaining_teams, p=probs)
            picks.append(chosen_team)
            remaining_teams.remove(chosen_team)
        
        for pick, team in enumerate(picks, start=1):
            results[team][pick] += 1
    
    return results

# Simulate the lottery 1 time (since we're just generating one draft order)
np.random.seed(None)  # Ensure a different seed for each run
simulation_results = simulate_lottery(probabilities, num_simulations=1)

# Get the draft order for the first 14 picks
draft_order = []
for pick in range(1, 15):
    for team, picks in simulation_results.items():
        if picks[pick] == 1:
            draft_order.append(team)

# Append the fixed order for picks 15-30
draft_order.extend(fixed_order)

# Display the full draft order
print("Draft order:")
for pick, team in enumerate(draft_order, start=1):
    print(f"{pick}: {team}")


Draft order:
1: Magic
2: Pelicans
3: Pacers
4: Hornets
5: Thunder
6: Pistons
7: Kings
8: Trailblazers
9: Spurs
10: Wizards
11: Knicks
12: Thunder (via Clippers)
13: 76ers
14: Cavaliers
15: Hornets (via Pelicans)
16: Hawks
17: Rockets (via Nets)
18: Bulls
19: Timberwolves
20: Spurs (via Raptors)
21: Nuggets
22: Grizzlies (via Jazz)
23: 76ers
24: Bucks
25: Spurs (via Celtics)
26: Rockets (via Mavericks)
27: Heat
28: Warriors
29: Grizzlies
30: Nuggets (via Thunder)


In [36]:
import pandas as pd
import numpy as np

# Load the team needs
nba_teams_path = 'NBA_Teams_Final_2022.csv'
nba_teams = pd.read_csv(nba_teams_path)

# Extract team needs
team_needs = {}
for index, row in nba_teams.iterrows():
    team = row['TEAM']
    needs = []
    for need in ['STAR POTENTIAL', 'SHOOTING', 'OFFENSE', 'DEFENSE', 'REBOUNDING', 'PLAYMAKING']:
        if row[need] == 'YES':
            needs.append(need.replace(" ", "_") + "_NEED")
    team_needs[team] = needs

# Load player statistics
player_stats_path = 'Player Stats 2022.csv'
player_stats = pd.read_csv(player_stats_path)

#intl_players_path = '2022 International Players(Sheet1).csv'
#intl_players = pd.read_csv(intl_players_path)

# Define weights for the star potential metric
weights = {
    'BPM': 0.20,
    'PTS': 0.15,
    'REB': 0.15,
    'AST': 0.15,
    'ORTG': 0.10,
    'DRTG': 0.10,
    'USG%': 0.10,
    'TS%': 0.05,
    'Year_Penalty': 0.05  # Penalty for being older
}

# Define penalties for the year
year_penalties = {
    'FR': 0,
    'SO': -0.05,
    'JR': -0.10,
    'SR': -0.15
}

# Function to calculate weighted star potential score
def calculate_star_potential(row, weights, year_penalties):
    weighted_sum = 0.0
    for metric, weight in weights.items():
        if metric != 'Year_Penalty' and metric in row.index:
            weighted_sum += row[metric] * weight
    year_penalty = year_penalties.get(row['Year'], 0) * weights['Year_Penalty']
    weighted_sum += year_penalty
    return weighted_sum

# Add a new column for the star potential score
player_stats['Star_Potential'] = player_stats.apply(lambda row: calculate_star_potential(row, weights, year_penalties), axis=1)

# Define weights for the shooting score metric
shooting_weights = {
    'TS%': 0.30,
    '3P%': 0.35,
    'eFG%': 0.25,
    'FT%': 0.10
}

# Function to calculate weighted shooting score
def calculate_shooting_score(row, shooting_weights):
    weighted_sum = 0.0
    for metric, weight in shooting_weights.items():
        if metric in row.index:
            weighted_sum += row[metric] * weight
    return weighted_sum

# Add a new column for the shooting score
player_stats['Shooting_Score'] = player_stats.apply(lambda row: calculate_shooting_score(row, shooting_weights), axis=1)

# Define weights for the offensive score metric
offensive_weights = {
    'OBPM': 0.40,
    'ORTG': 0.35,
    'PTS': 0.25
}

# Function to calculate weighted offensive score
def calculate_offensive_score(row, offensive_weights):
    weighted_sum = 0.0
    for metric, weight in offensive_weights.items():
        if metric in row.index:
            weighted_sum += row[metric] * weight
    return weighted_sum

# Add a new column for the offensive score
player_stats['Offensive_Score'] = player_stats.apply(lambda row: calculate_offensive_score(row, offensive_weights), axis=1)

# Define weights for the defensive score metric
defensive_weights = {
    'DBPM': 0.30,
    'DRTG': 0.25,
    'STL': 0.15,
    'BLK': 0.15,
    'D-PRPG': 0.15
}

# Function to calculate weighted defensive score
def calculate_defensive_score(row, defensive_weights):
    weighted_sum = 0.0
    for metric, weight in defensive_weights.items():
        if metric in row.index:
            weighted_sum += row[metric] * weight
    return weighted_sum

# Add a new column for the defensive score
player_stats['Defensive_Score'] = player_stats.apply(lambda row: calculate_defensive_score(row, defensive_weights), axis=1)

# Define weights for the rebounding score metric
rebounding_weights = {
    'OR': 0.5,
    'DR': 0.5
}

# Function to calculate weighted rebounding score
def calculate_rebounding_score(row, rebounding_weights):
    weighted_sum = 0.0
    for metric, weight in rebounding_weights.items():
        if metric in row.index:
            weighted_sum += row[metric] * weight
    return weighted_sum

# Add a new column for the rebounding score
player_stats['Rebounding_Score'] = player_stats.apply(lambda row: calculate_rebounding_score(row, rebounding_weights), axis=1)

# Define weights for the playmaking score metric
playmaking_weights = {
    'AST%': 0.4,
    'AST': 0.3,
    'A/TO': 0.3
}

# Function to calculate weighted playmaking score
def calculate_playmaking_score(row, playmaking_weights):
    weighted_sum = 0.0
    for metric, weight in playmaking_weights.items():
        if metric in row.index:
            weighted_sum += row[metric] * weight
    return weighted_sum

# Add a new column for the playmaking score
player_stats['Playmaking_Score'] = player_stats.apply(lambda row: calculate_playmaking_score(row, playmaking_weights), axis=1)

# Function to calculate draft value based on expected pick with higher weight
def calculate_draft_value(row, current_pick):
    expected_pick = row['PICK']
    return -abs(expected_pick * 1000000000000 - current_pick) * 1000 # Increase weight for expected pick


# Updated function to handle player drafting based on a specific stat
def draft_player(stat, current_pick, condition=None, drafted_players=None):
    sort1 = player_stats.sort_values(by=[stat], ascending=False)
    if condition is not None:
        sort1 = sort1.loc[condition]
    
    # Include players with a 'PICK' value even if they don't have stats
    players_with_pick = player_stats[player_stats['PICK'].notna()]
    sort1 = pd.concat([sort1, players_with_pick]).drop_duplicates(subset='Player').reset_index(drop=True)
    
    # Exclude drafted players
    if drafted_players:
        sort1 = sort1[~sort1['Player'].isin(drafted_players)]
        
    sort1['Draft_Value'] = sort1.apply(lambda row: calculate_draft_value(row, current_pick), axis=1)
    sort1['Combined_Score'] = sort1[stat].fillna(0) + sort1['Draft_Value']  # Fill NA values with 0 for scoring
    
    sort2 = sort1.sort_values(by='Combined_Score', ascending=False).reset_index(drop=True)
    sort3 = sort2.reset_index()
    return sort3


# Simulate the lottery (assuming `simulate_lottery` and `draft_order` are already defined)
# Replace draft_order with your generated draft order
draft_order = draft_order  # Replace with actual draft order from your simulation

# Store draft order for consistency
final_draft_order = list(draft_order)  # Store the draft order immediately after simulation

# Create an empty DataFrame to store drafted players
dfnba = pd.DataFrame()

# List to store the result
team_player_list = []
drafted_players = set()

# Iterate over the teams and their needs
for current_pick, team in enumerate(final_draft_order, start=1):
    needs = team_needs.get(team, ["STAR_POTENTIAL_NEED"])
    
    player_drafted = False
    for need in needs:
        if need == "STAR_POTENTIAL_NEED":
            sort3 = draft_player('Star_Potential', current_pick, drafted_players=drafted_players)
        elif need == "SHOOTING_NEED":
            sort3 = draft_player('Shooting_Score', current_pick, condition=player_stats['3P/100'] > 5, drafted_players=drafted_players)
        elif need == "OFFENSE_NEED":
            sort3 = draft_player('Offensive_Score', current_pick, drafted_players=drafted_players)
        elif need == "DEFENSE_NEED":
            sort3 = draft_player('Defensive_Score', current_pick, drafted_players=drafted_players)
        elif need == "REBOUNDING_NEED":
            sort3 = draft_player('Rebounding_Score', current_pick, drafted_players=drafted_players)
        elif need == "PLAYMAKING_NEED":
            sort3 = draft_player('Playmaking_Score', current_pick, drafted_players=drafted_players)
        else:
            print("Unexpected need type:", need)
            continue
        
        # Select the best available player with some variance
        if len(sort3) > 0:
            # Add variance by shuffling the top players before selection
            top_players = sort3.iloc[:min(len(sort3), 5)]  # Consider top 5 players for variance
            shuffled_players = top_players.sample(frac=0.6, replace=False).sort_values(by='Combined_Score', ascending=False)  # Shuffle slightly
            sort4 = shuffled_players.iloc[0]  # Select the first player after shuffling
            
            drafted_players.add(sort4['Player'])  # Add player to drafted players set
            dfnba = pd.concat([pd.DataFrame([sort4]), dfnba])
            dp_org = dfnba[['Player', 'Star_Potential', 'PTS', 'REB', 'AST', 'ORTG', 'DRTG', 'USG', 'TS', 'Shooting_Score', 'Offensive_Score', 'Defensive_Score', 'Rebounding_Score', 'Playmaking_Score']]
            dp_list = dp_org['Player'].tolist()
            college = sort4['TEAM']  # Capture college information here
            team_player_list.append((team, dp_list[0], college))  # Append team, player, and college
            
            # Remove the drafted player from the pool
            player_stats = player_stats[~(player_stats['Player'] == sort4['Player'])]
            player_stats = player_stats.reset_index(drop=True)
            
            # Move to the next team after drafting one player
            break

# Display the team and player pairing
for i, (team, player, college) in enumerate(team_player_list, start=1):
    if i == 1:
        print(f"With the \033[1m{i}st\033[0m pick in the 2022 NBA draft, the \033[1m{team}\033[0m select \033[1m{player}\033[0m from {college}.")
    elif i == 2:
        print(f"With the \033[1m{i}nd\033[0m pick in the 2022 NBA draft, the \033[1m{team}\033[0m select \033[1m{player}\033[0m from {college}.")
    elif i == 3:
        print(f"With the \033[1m{i}rd\033[0m pick in the 2022 NBA draft, the \033[1m{team}\033[0m select \033[1m{player}\033[0m from {college}.")
    else:
        print(f"With the \033[1m{i}th\033[0m pick in the 2022 NBA draft, the \033[1m{team}\033[0m select \033[1m{player}\033[0m from {college}.")


With the 1st pick in the 2022 NBA draft, the Magic select Paolo Banchero from Duke.
With the 2nd pick in the 2022 NBA draft, the Pelicans select Chet Holmgren from Gonzaga.
With the 3rd pick in the 2022 NBA draft, the Pacers select Jabari Smith from Auburn.
With the 4th pick in the 2022 NBA draft, the Hornets select Jaden Ivey from Purdue.
With the 5th pick in the 2022 NBA draft, the Thunder select Bennedict Mathurin from Arizona.
With the 6th pick in the 2022 NBA draft, the Pistons select Keegan Murray from Iowa.
With the 7th pick in the 2022 NBA draft, the Kings select Shaedon Sharpe from Kentucky.
With the 8th pick in the 2022 NBA draft, the Trailblazers select Dyson Daniels from  G League Ignite.
With the 9th pick in the 2022 NBA draft, the Spurs select Johnny Davis from Wisconsin.
With the 10th pick in the 2022 NBA draft, the Wizards select Ousmane Dieng from  New Zealand Breakers.
With the 11th pick in the 2022 NBA draft, the Knicks select Jalen Williams from Santa Clara.
With th